# Top

# [Test](#@Test)

In [1]:
_LCM = lambda f, g: f.parent().monomial_lcm(f, g)
_LC = lambda f : f.lc()
_LM = lambda f : f.lm()
_LT = lambda f : f.lt()
_Tail = lambda f : f - f.lt()


def choose(P):
    return max(P)


def ann(f):
    lc_f = _LC(f)
    if lc_f.is_unit() or lc_f.is_zero():
        return 0
    else:
        element = f.lc().parent().list()
        element.remove(0)
        for i in element:
            if lc_f * i == 0:
                return f.parent()(factor(Integer(i)).radical()[0][0])


def spoly(f, g):
    t = _LCM(f,g)
    t_f = t//_LM(f)
    t_g = t//_LM(g)
    
    a = lcm(_LC(f), _LC(g))
    a_f = a//_LC(f)
    a_g = a//_LC(g)
    return a_f*t_f*f - a_g*t_g*g


def gpoly(f, g):
    t = _LCM(f,g)
    t_f = t//_LM(f)
    t_g = t//_LM(g)
    
    b, b_f, b_g = xgcd(Integer(_LC(f)), Integer(_LC(g)))
    return b_f*t_f*f + b_g*t_g*g


def apoly(f):
    a = ann(f)
    return a*_Tail(f)

# Buchberger Algorithm

In [2]:
def buchberger(F):

    G = set(F.gens())
    B = set((g1, g2) for g1 in G for g2 in G if g1 != g2)

    while B:
        g1, g2 = select(B)
        
        for g in g1,g2:
            if not _LC(g).is_unit():
                return _LC(g)
        
        B.remove((g1, g2))

        h = spoly(g1, g2).reduce(G)
        if h != 0:
            B = B.union((g, h) for g in G)
            G.add(h)


    return Sequence(G).reduced()

def select(P):
    return min(P, key=lambda fi_fj: _LCM(_LM(fi_fj[0]),
                                        _LM(fi_fj[1])).total_degree())


# ! Proposisi 16 : Case 2 only

In [3]:
def factorize(n):
    factor_n = factor(n)
    x, y = factor_n[0]
    q = n/(x^y)
    return (x^y), q

# ! Corolary 14

In [4]:
def fki(g1, g2, u, v, a, b, R):
    idtmp1 = R.ideal(g1)
    idtmp2 = R.ideal(g2)
    rg1 = Sequence(idtmp1)    
    rg2 = Sequence(idtmp2)
    rg1.append(a)
    rg2.append(b)
    G = set()
    for k in rg1:
        for l in rg2:
            f = (u*a*_LCM(_LM(k),_LM(l))*_LC(k)*l//_LM(l)) + (v*b*_LCM(_LM(k),_LM(l))*_LC(l)*k//_LM(k))
            G = G.union({f})
    return Sequence(G).reduced()

# ! Algoritma Eder Hofmann ft. Buchberger

In [5]:
def GB_PIR_BBGR(I):
    R = I.ring()
    Rn = R.base_ring()
    
    if Rn.is_field():
        
        return buchberger(I)
    else:

        factor_n = factor(Rn.cardinality())
        if len(factor_n) == 1:
            
            
            return I.groebner_basis('singular').reduced()
        else:
            
            buchberger_test = buchberger(I)
            if isinstance(buchberger_test, list):
                
                return buchberger_test
                
            else:
                p, q = factorize(Rn.cardinality())
                Rp = I.ring().change_ring(Integers(p))
                Ip = Rp.ideal(Sequence(Rp.ideal(I).gens()).reduced())
                Gbp = GB_PIR_BBGR(Ip)
                Rq = I.ring().change_ring(Integers(q))
                Iq = Rq.ideal(Sequence(Rq.ideal(I).gens()).reduced())
                Gbq = GB_PIR_BBGR(Iq)
                
                d, u, v = xgcd(p,q)

                G = fki(Gbp, Gbq, R(u),R(v),R(p),R(q),R)

                return G.reduced()
        
        
        
#

# @Test

In [6]:
R.<a,b,c,d,e,f,g,h,i> = PolynomialRing(Integers(900))
R

Multivariate Polynomial Ring in a, b, c, d, e, f, g, h, i over Ring of integers modulo 900

In [34]:
# F = sage.rings.ideal.Katsura(R,8)
F =  R.ideal(a + a*b + b*c + c*d + d*e + e*f + f*g -   h, b + a*c + b*d + c*e + d*f + e*g - 2*h,	c + a*d + b*e + c*f + d*g - 3*h, d + a*e + b*f + c*g - 4*h, e + a*f + b*g - 5*h, f + a*g - 6*h, g - 7*h, a + b + c + d + e + f + g + 1)
F

Ideal (a*b + b*c + c*d + d*e + e*f + f*g + a + 899*h, a*c + b*d + c*e + d*f + e*g + b + 898*h, a*d + b*e + c*f + d*g + c + 897*h, a*e + b*f + c*g + d + 896*h, a*f + b*g + e + 895*h, a*g + f + 894*h, g + 893*h, a + b + c + d + e + f + g + 1) of Multivariate Polynomial Ring in a, b, c, d, e, f, g, h, i over Ring of integers modulo 900

In [35]:
%time GB_PIR_BBGR(F)

CPU times: user 14.4 s, sys: 23.7 ms, total: 14.4 s
Wall time: 14.6 s


Polynomial Sequence with 65 Polynomials in 8 Variables

In [53]:
%time GB_PIR_BBGR(F) # eco 9

CPU times: user 18min 16s, sys: 1.07 s, total: 18min 17s
Wall time: 18min 17s


Polynomial Sequence with 66 Polynomials in 9 Variables

# [Top](#Top)